# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [2]:
# import required libraries
import numpy as np
import pandas as pd

Next, extract and import `Orders` dataset into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [8]:
#Extract and import Orders
orders=pd.read_csv('Orders.csv')

#Remove first column. It's equal to index
orders.drop(columns=orders.columns[0], axis=1, inplace=True)

orders.head()

,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34


In [39]:
orders.shape

(397924, 13)

---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [113]:
#Aggregate amount_spent for each customer
aggregation=orders.groupby(['CustomerID'], as_index=False).agg({'amount_spent':'sum'})
aggregation

,CustomerID,amount_spent
0,12346,77183.60
1,12347,4310.00
2,12348,1797.24
3,12349,1757.55
4,12350,334.40
...,...,...
4334,18280,180.60
4335,18281,80.82
4336,18282,178.05
4337,18283,2094.88


In [114]:
#Amount spent in 75th and 95th percentile
q95=aggregation['amount_spent'].quantile(.75)
q75=aggregation['amount_spent'].quantile(.95)


vip = aggregation[aggregation['amount_spent'] > aggregation['amount_spent'].quantile(.95)]

preferred = aggregation[(aggregation['amount_spent'].quantile(.75) < aggregation['amount_spent']) & (aggregation['amount_spent'] < aggregation['amount_spent'].quantile(.95))]


In [115]:
vip

,CustomerID,amount_spent
0,12346,77183.60
10,12357,6207.67
12,12359,6372.58
50,12409,11072.67
55,12415,124914.53
...,...,...
4207,18109,8052.97
4229,18139,8438.34
4253,18172,7561.68
4292,18223,6484.54


In [116]:
preferred

,CustomerID,amount_spent
1,12347,4310.00
2,12348,1797.24
3,12349,1757.55
5,12352,2506.04
9,12356,2811.43
...,...,...
4319,18259,2338.60
4320,18260,2643.20
4328,18272,3078.58
4337,18283,2094.88


Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [118]:
#Aggregate amount_spent for unique customers and BY COUNTRY
aggregation=orders.groupby(['CustomerID','Country'], as_index=False).agg({'amount_spent':'sum'})
#Amount spent in 75th and 95th percentile
q95=aggregation['amount_spent'].quantile(.75)
q75=aggregation['amount_spent'].quantile(.95)


vip = aggregation[aggregation['amount_spent'] > aggregation['amount_spent'].quantile(.95)]

preferred = aggregation[(aggregation['amount_spent'].quantile(.75) < aggregation['amount_spent']) & (aggregation['amount_spent'] < aggregation['amount_spent'].quantile(.95))]


top_vip = vip.groupby(['Country'], as_index=False).agg({'CustomerID':'count'})
max_customer = top_vip['CustomerID'].max()
top_country = top_vip[top_vip['CustomerID'] == max_customer]['Country'].tolist()[0]
print(f"The country with most VIP Customer is {top_country} with {max_customer}.")

The country with most VIP Customer is United Kingdom with 178.


## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [101]:
combined = pd.concat([vip, preferred]) # appending the rows of preferred below the rows of vip
combined['Country'].value_counts()

Country
United Kingdom     934
Germany             39
France              29
Belgium             11
Switzerland          9
Norway               7
Spain                7
Portugal             7
Finland              5
Italy                5
Australia            4
Channel Islands      4
Japan                4
EIRE                 3
Denmark              3
Cyprus               3
Sweden               2
Israel               2
Netherlands          1
Singapore            1
Iceland              1
Austria              1
Lebanon              1
Poland               1
Greece               1
Canada               1
Malta                1
Name: count, dtype: int64

In [119]:
combined

,CustomerID,Country,amount_spent
0,12346,United Kingdom,77183.60
10,12357,Switzerland,6207.67
12,12359,Cyprus,6372.58
52,12409,Switzerland,11072.67
57,12415,Australia,124914.53
...,...,...,...
4327,18259,United Kingdom,2338.60
4328,18260,United Kingdom,2643.20
4336,18272,United Kingdom,3078.58
4345,18283,United Kingdom,2094.88


In [108]:
top_customers = combined['Country'].iloc[0]

In [109]:
top_customers

'United Kingdom'